In [68]:
import pandas as pd
import os, re
from pathlib import Path
import numpy
import requests
import slugify
from dateutil.parser import *
import numpy as np

In [69]:
prefix = 
"""@prefix  rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix  rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix  owl: <http://www.w3.org/2002/07/owl#>.
@prefix  xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix  foaf: <http://xmlns.com/foaf/0.1/>.
@prefix  halalv: <http://halal.addi.is.its.ac.id/halalv.ttl#>.
@prefix  halalf: <http://halal.addi.is.its.ac.id/foodproducts/>.
@prefix  halali: <http://halal.addi.is.its.ac.id/ingredients/>.
@prefix  halalc: <http://halal.addi.is.its.ac.id/certificates/>.
@prefix  halals: <http://halal.addi.is.its.ac.id/sources/>.
@prefix  halalm: <http://halal.addi.is.its.ac.id/manufactures/>.
@prefix  food: <http://purl.org/foodontology#> .
@prefix  foodlirmm: <http://data.lirmm.fr/ontologies/food#> .
@prefix  gr: <http://purl.org/goodrelations/v1#> . 
@prefix  dbr: <http://dbpedia.org/resource/> .

"""

In [70]:
def formatDateX(string):
    try:
        date = parse(string)
        date = date.strftime('%Y-%m-%d')
        return str(date)
    except:
        return "0000-00-00"

In [71]:
def makeSlug(string):
    slug = slugify.slugify(string)
    slug = slug.replace("-", " ")
    slug = slug.title()
    slug = slug.replace(" ", "_")
    return slug

In [72]:
def getManufactureName(string):
    print("Value string manufacture: " + str(string))
    try:
        product = str(string).split(",", 1)[0].title()
    except:
        product = str(string).title()
    print("Value now string manufacture: " + product)
    return product

In [73]:
def getProductName(string):
    print("Value string product: " + str(string))
    try:
        product = str(string).split(",", 1)[1].title()
    except:
        product = str(string).title()
    print("Value now string product: " + product)
    return product

In [74]:
def getIngredients(string):
    ingredients = str(string).title()
    return ingredients

In [75]:
def getCertTrim(string):
    cert = str(string).strip()
    return cert

In [76]:
def getTrimAndTitle(string):
    newstring = str(string).strip()
    newstring = newstring.title()
    return newstring

In [77]:
def getUniqueIngredients(list):
    ingredientList = []
    for data in list:
        #print("Data: "+data)
        for ingredient in str(data).split(","):
            #print("Get ingredient: "+ingredient)
            ingredient = ingredient.strip()
            ingredientList.append(ingredient)
    #newingredientList = list(dict.fromkeys(ingredientList))
    return ingredientList

In [78]:
def createIngredientTTL(ingredients, path):

    noOwlSameAs = """halali:{slug} a foodlirmm:Ingredient;
   rdfs:label "{name}" .

   """

    withOwlSameAs = """halali:{slug} a foodlirmm:Ingredient;
   rdfs:label "{name}";
   owl:sameAs dbr:{slug} .

   """
    for ingredient in ingredients:
        print("Creating ingredient: " + str(ingredient))
        slugIngredient = slugify.slugify(ingredient)
        slugIngredient = str(slugIngredient).replace("-", " ")
        slugIngredient = slugIngredient.title()
        slugIngredient = slugIngredient.replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","")
    # url = "https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=ASK+%7B%0D%0A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F%7Bslug%7D%3E+%3Fp+%3Fo+.%0D%0A%7D&format=text%2Fhtml&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+%22.format(  
    #     slug=slugIngredient)
    # r = requests.get(url)
    # data = r.text
    # if "true" in data:
    #     print("Get owl:sameAs")
    #     withOwlSameAs = withOwlSameAs.format(name=ingredient, slug=slugIngredient)
    #     f = open("data\\dat\\new\\jadi\\noprefix\\ingredients\\" + slugIngredient + ".ttl", "a+", encoding="utf-8")
    #     f.write(withOwlSameAs)
    #     f.close()
    #     withOwlSameAsPrefix = prefix + withOwlSameAs
    #     f = open("data\\dat\\new\\jadi\\withprefix\\ingredients\\" + slugIngredient + ".ttl", "a+", encoding="utf-8")
    #     f.write(withOwlSameAsPrefix)
    #     f.close()
    # else:
    #     print("Doesn't get owl:sameAs")
    #     noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
    #     f = open("data\\dat\\new\\jadi\\noprefix\\ingredients\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    #     f.write(noOwlSameAs)
    #     f.close()
    #     noOwlSameAsPrefix = prefix + noOwlSameAs
    #     f = open("data\\dat\\new\\jadi\\withprefix\\ingredients\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    #     f.write(noOwlSameAsPrefix)
    #     f.close()

        pathFolderNoPrefix = path + "\\resources\\noprefix\\ingredients"
        pathFolderWithPrefix = path + "\\resources\\withprefix\\ingredients"

        if not os.path.isdir(pathFolderNoPrefix):
            os.makedirs(pathFolderNoPrefix)

        if not os.path.isdir(pathFolderWithPrefix):
            os.makedirs(pathFolderWithPrefix)

        # print("Doesn't get owl:sameAs")
        noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
        f = open(pathFolderNoPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
        f.write(noOwlSameAs)
        f.close()
        noOwlSameAsPrefix = prefix + noOwlSameAs
        f = open(pathFolderWithPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
        f.write(noOwlSameAsPrefix)
        f.close()

In [79]:
def createIngredientSeparate(ingredient, path):
    ingText = """halali:{sluging} a foodlirmm:Ingredient;
   rdfs:label "{nameing}" .

   """
    
    ingredient = str(ingredient).replace('"','').replace("?","").replace("$","").replace("(e)","").replace("(","").replace(")","").replace("%","").replace(".","").replace("\\","").replace("&","")
    print("Creating ingredient: " + str(ingredient))
    slugIngredient = slugify.slugify(ingredient)
    slugIngredient = str(slugIngredient).replace("-", " ")
    slugIngredient = slugIngredient.title()
    slugIngredient = slugIngredient.replace(" ", "_").replace("%","").replace("$","").replace("(","").replace(")","")
        

    pathFolderNoPrefix = path + "\\resources\\noprefix\\ingredients"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\ingredients"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

        # print("Doesn't get owl:sameAs")
        #noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
    ingText = ingText.format(sluging=slugIngredient, nameing=ingredient)
    f = open(pathFolderNoPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    f.write(ingText)
    f.close()
    noOwlSameAsPrefix = prefix + ingText
    f = open(pathFolderWithPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    f.write(noOwlSameAsPrefix)
    f.close()

In [80]:
def createManufactureTTL(manufacture, path):
    manufactureText = """halalm:{slug} a foaf:Organization;
		rdfs:label "{name}" .

		"""
    slugManufacture = slugify.slugify(manufacture)
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")
    manufacture = str(manufacture).replace('"','').replace("?","").replace("$","").replace("(","").replace(")","").replace("\\","").replace("&","")
    manufactureText = manufactureText.format(slug=slugManufacture, name=manufacture)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\manufactures"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\manufactures"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    f = open(pathFolderNoPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureText)
    f.close()
    manufactureTextPrefix = prefix + manufactureText
    f = open(pathFolderWithPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureTextPrefix)
    f.close()

In [81]:
def createManufactureTTLSusulan(manufacture, path):
    manufactureText = """halalm:{slug} a foaf:Organization;
		rdfs:label "{name}" .

		"""
    slugManufacture = slugify.slugify(manufacture)
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")
    manufacture = str(manufacture).replace('"','').replace("?","").replace("$","").replace("(","").replace(")","").replace("\\","").replace("&","")
    manufactureText = manufactureText.format(slug=slugManufacture, name=manufacture)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\manufactures"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\manufactures"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    f = open(pathFolderNoPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureText)
    f.close()
    manufactureTextPrefix = prefix + manufactureText
    f = open(pathFolderWithPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureTextPrefix)
    f.close()

In [82]:
def createCertificateTTL(row, path):
    certString = """halalc:{slugCertificate} a halalv:HalalCertificate;
			halalv:halalCode "{certificate}";
			halalv:halalExp "{expired}"^^xsd:date;
			halalv:OrgCert halals:{slugOrg} .

			"""

    orgString = """halals:{slugOrganization} a foaf:Organization;
    rdfs:label "{orgName}";
    foaf:homepage "{cert_bodyname}";
    foaf:based_near {location} .

    """

    expired = formatDateX(str(row['expired']))

    certCode = str(row['cert_code'])
    if ('0' == str(row['cert_code'])):
        certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
    elif ('nan' == str(row['cert_code'])):
        certCode = str(row['cert_bodyname']).upper()
    else:
        certCode = str(row['cert_code']).upper()

    slugCertificate = slugify.slugify(certCode)
    slugCertificate = str(slugCertificate).replace("-", "_")
    slugCertificate = slugCertificate.upper()

    slugOrganization = slugify.slugify(str(row['cert_bodyname']))
    slugOrganization = str(slugOrganization).replace("-", " ")
    slugOrganization = slugOrganization.title()
    slugOrganization = str(slugOrganization).replace(" ", "_")

    certString = certString.format(slugCertificate=slugCertificate, certificate=str(row['cert_code']),
                                   expired=str(expired), slugOrg=slugOrganization)
    orgString = orgString.format(slugOrganization=slugOrganization, orgName=str(row['cert_bodyname']),
                                 cert_bodyname=str(row['homepage']), location=str(row['location']))

    pathFolderNoPrefix = path + "\\resources\\noprefix\\certificates"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\certificates"
    pathFolderNoPrefixOrg = path + "\\resources\\noprefix\\halalsources"
    pathFolderWithPrefixOrg = path + "\\resources\\withprefix\\halalsources"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    if not os.path.isdir(pathFolderNoPrefixOrg):
        os.makedirs(pathFolderNoPrefixOrg)

    if not os.path.isdir(pathFolderWithPrefixOrg):
        os.makedirs(pathFolderWithPrefixOrg)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + slugCertificate
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + slugCertificate
    pathFileNoPrefixOrg = pathFolderNoPrefixOrg + "\\" + slugOrganization
    pathFileWithPrefixOrg = pathFolderWithPrefixOrg + "\\" + slugOrganization

    f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certString)
    f.close()
    certStringPrefix = prefix + certString
    f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certStringPrefix)
    f.close()

    f = open(pathFileNoPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgString)
    f.close()
    orgStringPrefix = prefix + orgString
    f = open(pathFileWithPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgStringPrefix)
    f.close()

In [83]:
def createCertificateTTLSusulan(row, path):
    certString = """halalc:{slugCertificate} a halalv:HalalCertificate;
			halalv:halalCode "{certificate}";
			halalv:halalExp "{expired}"^^xsd:date;
			halalv:OrgCert halals:{slugOrg};
			halalv:halalStatus {status} .

			"""

    orgString = """halals:{slugOrganization} a foaf:Organization;
    rdfs:label "{orgName}";
    foaf:homepage "{cert_bodyname}";
    foaf:based_near {location} .

    """

    expired = formatDateX(str(row['expired']))

    certCode = str(row['cert_code'])
    if ('0' == str(row['cert_code'])):
        certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
    elif ('nan' == str(row['cert_code'])):
        certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
    else:
        certCode = str(row['cert_code']).upper()

    certStatus = "halalv:"+str(row['status'])

    slugCertificate = slugify.slugify(certCode)
    slugCertificate = str(slugCertificate).replace("-", "_")
    slugCertificate = slugCertificate.upper()

    slugOrganization = slugify.slugify(str(row['cert_bodyname']))
    slugOrganization = str(slugOrganization).replace("-", " ")
    slugOrganization = slugOrganization.title()
    slugOrganization = str(slugOrganization).replace(" ", "_")

    certString = certString.format(slugCertificate=slugCertificate, certificate=str(row['cert_code']),
                                   expired=str(expired), slugOrg=slugOrganization, status=certStatus)
    orgString = orgString.format(slugOrganization=slugOrganization, orgName=str(row['cert_bodyname']),
                                 cert_bodyname=str(row['homepage']), location=str(row['location']))

    pathFolderNoPrefix = path + "\\resources\\noprefix\\certificates"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\certificates"
    pathFolderNoPrefixOrg = path + "\\resources\\noprefix\\halalsources"
    pathFolderWithPrefixOrg = path + "\\resources\\withprefix\\halalsources"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    if not os.path.isdir(pathFolderNoPrefixOrg):
        os.makedirs(pathFolderNoPrefixOrg)

    if not os.path.isdir(pathFolderWithPrefixOrg):
        os.makedirs(pathFolderWithPrefixOrg)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + slugCertificate
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + slugCertificate
    pathFileNoPrefixOrg = pathFolderNoPrefixOrg + "\\" + slugOrganization
    pathFileWithPrefixOrg = pathFolderWithPrefixOrg + "\\" + slugOrganization

    f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certString)
    f.close()
    certStringPrefix = prefix + certString
    f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certStringPrefix)
    f.close()

    f = open(pathFileNoPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgString)
    f.close()
    orgStringPrefix = prefix + orgString
    f = open(pathFileWithPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgStringPrefix)
    f.close()

In [84]:
def createProductTTL(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .

halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

    halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(".", "").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [85]:
def createProductTTLSusulan(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .

halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

    halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [86]:
def createProductTTLSusulanCoba(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .
    foodlirmm:certificate halalc:{slugCertificate} .
    food:containsIngredient {concateIngredientsEntity} .
    food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .
        food:containsIngredient {concateIngredientsEntity} .
        food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .
        foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [87]:
def execCreateProductTTL(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTL(row, path)

In [88]:
def execCreateProductTTLSusulan(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTLSusulan(row, path)

In [89]:
def execCreateProductTTLSusulanCoba(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTLSusulanCoba(row, path)

In [90]:
def countUniqueValuesEachEntity(df):
    print("Unique values: " + str(len(df['name'].unique().tolist())))

    # Select all duplicate rows based on one column
    duplicateRowsDF = df[df.duplicated(['name'])]
    duplicateRowsDF = duplicateRowsDF.sort_values(by=['name'])
    print("Duplicate rows length: " + str(len(duplicateRowsDF)))
    for index, row in duplicateRowsDF.iterrows():
        print("Values are: " + str(row['code']) + " - " + str(row['name']) + " - " + str(
            row['manufacture']) + " - " + str(row['cert_code']) + " - " + str(row['expired']) + " - " + str(
            row['cert_bodies']) + " - " + str(row['cert_bodyname']) + " - " + str(row['homepage']) + " - " + str(
            row['location']) + " - " + str(row['ingredients']))

In [91]:
def execCreateCertAndOrganizationTTL(df, path):
    dfcert = df[['cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location']].copy()
    # dfcert = dfcert.dropna()
    # #
    # # df = df.drop_duplicates()
    #
    # for name in dfcert.cert_bodyname.unique():
    #     print("Values is: "+str(name))
    for index, row in dfcert.iterrows():
        if 'nan' == str(row['cert_bodies']):
            print("Skipping, no cert bodyname..")
            pass
        else:
            print("Creating certificate on: " + str(row['cert_code']) + str(row['cert_bodies']))
            createCertificateTTL(row, path)

In [92]:
def execCreateCertAndOrganizationTTLSusulan(df, path):
    dfcert = df[['cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location', 'status']].copy()
    # dfcert = dfcert.dropna()
    # #
    # # df = df.drop_duplicates()
    #
    # for name in dfcert.cert_bodyname.unique():
    #     print("Values is: "+str(name))
    for index, row in dfcert.iterrows():
        #if 'nan' == str(row['cert_bodies']):
            #print("Skipping, no cert bodyname..")
            #pass
        #else:
            print("Creating certificate on: " + str(row['cert_code']) )##+ str(row['cert_bodies']))
            createCertificateTTLSusulan(row, path)

In [93]:
def execCreateIngredientsTTLFromFile(df, path):
    with open('ingredients.txt', 'r') as f:
        lines = f.readlines()
        for ingredient in lines:
            ingredient =  ingredient.replace("\n","")
            print("Working on: "+ingredient)
            createIngredientTTL(ingredient, path)

In [94]:
def execCreateIngredientsTTLFromDF(df, path):
    # create ingredients
    listIngredient = df['ingredients'].dropna().tolist()
    ingredients = getUniqueIngredients(listIngredient)
    newlistIng = list(dict.fromkeys(ingredients))
    createIngredientTTL(ingredients, path)

In [95]:
def execCreateIngredientsTTLFromDFCoba(df, path):
    # create ingredients
    listIngredient = df['ingredients'].dropna().tolist()
    ingredients = getUniqueIngredients(listIngredient)
    newlistIng = list(dict.fromkeys(ingredients))
    for ingredient in newlistIng:
        createIngredientSeparate(ingredient, path)

In [96]:
def execCreateManufacturerTTL(df, path):
    df = df.sort_values(by ='manufacture' )
    dfObj = df['manufacture']
    for manufacture in dfObj.unique().tolist():

        if (str(manufacture) == 'nan'):
            pass
        else:
            print("Working on: " + str(manufacture))
            createManufactureTTL(manufacture, path)

In [97]:
def execCreateManufacturerTTLSusulan(df, path):
    df = df.sort_values(by ='manufacture' )
    dfObj = df['manufacture']
    for manufacture in dfObj.unique().tolist():

        if (str(manufacture) == 'nan'):
            pass
        else:
            print("Working on: " + str(manufacture))
            createManufactureTTLSusulan(manufacture, path)

In [98]:
# url = "https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=ASK+%7B%0D%0A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F{slug}%3E+%3Fp+%3Fo+.%0D%0A%7D&format=text%2Fhtml&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+".format(
#     slug="Monosodium_glutamate")
# r = requests.get(url)
# data = r.text
# print("The content is: "+data)
# if "true" in data:
#     print("Get owl:sameAs")
# else:
#     print("Doesn't get owl:sameAs")

# slugIngredient = slugify.slugify("aku apa")
# print(slugIngredient)

#     createIngredientTTL()
# df['manufacture'] = df['manufacture'].apply(lambda x: getTrimAndTitle(x))
# df['name'] = df['name'].apply(lambda x: getTrimAndTitle(x))
# dforgs = df[df['cert_bodyname'] == numpy.nan]
# for dforg in dforgs:
#     print("Value of nan: "+str(len(dforg)))
# print("Size before drop mui: " + str(len(df)) + " - size unique org cert: " + str(len(df.cert_bodyname.unique())))
# df = df[~df['cert_bodyname'].isin(['https://www.halalmui.org/'])]
# print("Size after: " + str(len(df)) + " - size unique omanufacture: "+str(len(df.manufacture.unique()))+" - name "+str(len(df.name.unique())))
# for sert in df.manufacture.unique():
#     print("Sert: "+str(sert))

# for index, row in df.iterrows():
#     print("Value is: name: "+str(row['name'])+" - manufacture: "+str(row['manufacture']))

# df = pd.read_csv(path+".csv", delimiter=";", error_bad_lines=False,names=['id', 'code', 'name', 'verify', 'ingredients'])
# newdf = df[['code']].copy()
#
# newdf['name'] = df['name'].apply(lambda x: getProductName(x))
# newdf['manufacture'] = df['name'].apply(lambda x: getManufactureName(x))
# print("Length code: "+str(len(newdf['code']))+" - name: "+str(len(newdf['name']))+" - manufacture: "+str(len(newdf['manufacture'])))
# newdf['ingredientsx'] = ''
# newdf['ingredients1'] = ''
# newdf['ingredients2'] = ''
# newdf['ingredients3'] = ''
# newdf['ingredients4'] = ''
# newdf['ingredients5'] = ''
# newdf['ingredients'] = df['ingredients'].apply(lambda x: getIngredients(x))
# for data in newdf['name']:
#     print(str(data))
# for index, row in df.iterrows():
#     # print(str(row['code'])+" - "+str(row['name'])+" - "+str(row['ingredients']))
#     print(str(row['code'])+" - "+str(row['name'])+" - "+str(row['ingredients']))
# df['ingredients'] = ''
# df['ingredients1'] = ''
# df['ingredients2'] = ''
# df['ingredients3'] = ''
# df['ingredients4'] = ''
# df['ingredients5'] = ''
# df.to_csv(path+"_ready.csv", sep=";", index=None, header=None)

In [105]:
path = "datanew"
filename = "Hasil Benar7"
df = pd.read_csv(path + "\\" + filename + ".csv", delimiter=",", error_bad_lines=False, low_memory=False)
df
# ,names=['code', 'name', 'manufacture', 'cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location', 'ingredients', 'status']
                     

,code,name,produkmui,manufacture,cert_code,nilai,label,ingredients,ingredients_lama,ing0,...,ing29,ing30,ing31,ing32,cert_bodies,expired,cert_bodyname,homepage,location,status
0,0,Dua Kelinci Kacang Sangrai,Dua kelinci kacang sangrai,Pt. dua kelinci,00100017571001,"0,8",1,"kacang tanah,minyak nabati,garam,penguat rasa,...","kacang tanah, minyak nabati, garam, penguat ra...",kacang tanah,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
1,1,Dua Kelinci Kacang Polong Polongmas Ayam Bawang,Dua kelinci polongmas kacang polong rasa ayam ...,Pt. dua kelinci,00100017571001,"0,7777777778",1,"kacang polong,minyak sawit,bumbu ayam bawang,p...","kacang polong (.%), minyak sawit (mengandung a...",kacang polong,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
2,2,Dua Kelinci Jagung Presto Bawang Putih,Dua kelinci jagung presto rasa bawang putih,Pt. dua kelinci,00100017571001,"0,75",1,"jagung,minyak sawit,bumbu bawang putih,gula,di...","""jagung (.%), minyak sawit (mengandung antioks...",jagung,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
3,3,Frisian Flag Susu Bubuk Full Cream,Frisian flag susu bubuk full cream,Pt. frisian flag indonesia,00040040520606,"0,75",1,"susu bubuk instant full cream,mineral","susu bubuk instant full cream, mineral.",susu bubuk instant full cream,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
4,4,Dua Kelinci Jagung Presto Pedas Manis,Dua kelinci jagung presto rasa pedas manis,Pt. dua kelinci,00100017571001,"0,75",1,"jagung,minyak sawit,gula,dinatrium guanilat,di...","jagung (.%), minyak sawit (mengandung antioksi...",jagung,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,1191,Buavita Guava,Buavita,Pt. eka bogainti,00160048830908,"0,2",1,"air,konsentrat jambu,sukrosa,perisa identik al...","air, konsentrat jambu (.%), sukrosa, perisa id...",air,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
535,1198,Meg Keju Serbaguna 10'S,Meg slice serbaguna,Pt. megmilk snow brand indonesia,00040065560613,"0,2",1,kosong,kosong,kosong,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
536,1202,Fruitamin Minuman Coco Bit Splash Coco,Fruitamin cocobit minuman nata de coco rasa leci,Pt. anugerah indofood barokah makmur,00120038970306,"0,2",1,"air,gula,nata de coco,fruktosa,natrium karboks...","air, gula, nata de coco %, fruktosa, penstabil...",air,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
537,1210,Donald Yogu 10'S,Donald yogu puding rasa anggur,"Makmur cipta pangan, pt",17250055780920,"0,2",1,kosong,kosong,kosong,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal


In [106]:
NaN = np.nan
df["cert_bodies"] = NaN
df["expired"] = NaN
df["cert_bodyname"] = "Majelis Ulama Indonesia"
df["homepage"] = NaN
df["location"] = NaN
df["ingredients"] = df["ingredients"].replace('kosong', np.nan , regex=True)
df["ingredients"] = df["ingredients"].replace(',', ', ' , regex=True)
#df["ingredients"] = df["ingredients"].replace('.', '' , regex=True)
df["status"] = "Halal"
df["name"] = df["name"].str.lower()
df["manufacture"] = df["manufacture"].str.lower()
df["ingredients"] = df["ingredients"].str.lower()
##df = df[df.label == 0]
##df
##df.to_csv("Hasil Salah.csv")
df

,code,name,produkmui,manufacture,cert_code,nilai,label,ingredients,ingredients_lama,ing0,...,ing29,ing30,ing31,ing32,cert_bodies,expired,cert_bodyname,homepage,location,status
0,0,dua kelinci kacang sangrai,Dua kelinci kacang sangrai,pt. dua kelinci,00100017571001,"0,8",1,"kacang tanah, minyak nabati, garam, penguat ra...","kacang tanah, minyak nabati, garam, penguat ra...",kacang tanah,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
1,1,dua kelinci kacang polong polongmas ayam bawang,Dua kelinci polongmas kacang polong rasa ayam ...,pt. dua kelinci,00100017571001,"0,7777777778",1,"kacang polong, minyak sawit, bumbu ayam bawang...","kacang polong (.%), minyak sawit (mengandung a...",kacang polong,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
2,2,dua kelinci jagung presto bawang putih,Dua kelinci jagung presto rasa bawang putih,pt. dua kelinci,00100017571001,"0,75",1,"jagung, minyak sawit, bumbu bawang putih, gula...","""jagung (.%), minyak sawit (mengandung antioks...",jagung,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
3,3,frisian flag susu bubuk full cream,Frisian flag susu bubuk full cream,pt. frisian flag indonesia,00040040520606,"0,75",1,"susu bubuk instant full cream, mineral","susu bubuk instant full cream, mineral.",susu bubuk instant full cream,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
4,4,dua kelinci jagung presto pedas manis,Dua kelinci jagung presto rasa pedas manis,pt. dua kelinci,00100017571001,"0,75",1,"jagung, minyak sawit, gula, dinatrium guanilat...","jagung (.%), minyak sawit (mengandung antioksi...",jagung,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,1191,buavita guava,Buavita,pt. eka bogainti,00160048830908,"0,2",1,"air, konsentrat jambu, sukrosa, perisa identik...","air, konsentrat jambu (.%), sukrosa, perisa id...",air,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
535,1198,meg keju serbaguna 10's,Meg slice serbaguna,pt. megmilk snow brand indonesia,00040065560613,"0,2",1,NaN,kosong,kosong,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
536,1202,fruitamin minuman coco bit splash coco,Fruitamin cocobit minuman nata de coco rasa leci,pt. anugerah indofood barokah makmur,00120038970306,"0,2",1,"air, gula, nata de coco, fruktosa, natrium kar...","air, gula, nata de coco %, fruktosa, penstabil...",air,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
537,1210,donald yogu 10's,Donald yogu puding rasa anggur,"makmur cipta pangan, pt",17250055780920,"0,2",1,NaN,kosong,kosong,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal


In [107]:
execCreateProductTTLSusulan(df, path)
execCreateCertAndOrganizationTTLSusulan(df, path)
execCreateManufacturerTTLSusulan(df, path)
execCreateIngredientsTTLFromDFCoba(df, path)

Creating product ttl on: dua kelinci kacang sangrai
Creating product ttl on: dua kelinci kacang polong polongmas ayam bawang
Creating product ttl on: dua kelinci jagung presto bawang putih
Creating product ttl on: frisian flag susu bubuk full cream
Creating product ttl on: dua kelinci jagung presto pedas manis
Creating product ttl on: frisian flag krimer kental manis omela
Creating product ttl on: good family biscuit assorted
Creating product ttl on: van houten cocoa powder
Creating product ttl on: ottogi honey citron tea
Creating product ttl on: delfi hot cocoa indulgence
Creating product ttl on: cadbury hot chocolate drink 3 in 1
Creating product ttl on: ceres baking heritage chocolate dark choco chip
Creating product ttl on: frisian flag susu kental manis coklat
Creating product ttl on: dua kelinci kacang sukro oven bawang
Creating product ttl on: ceres baking heritage chocolate milk choco chip
Creating product ttl on: deka mini wafer bites durian
Creating product ttl on: deka mini 

Creating product ttl on: sosro teh botol jasmine tea
Creating product ttl on: espe larutan penyegar orange
Creating product ttl on: espe larutan penyegar leci
Creating product ttl on: kongbap multi grain mix 6's
Creating product ttl on: espe larutan penyegar strawberry
Creating product ttl on: roma crackers malkist cokelat kelapa
Creating product ttl on: kapal api coffee drink white coffee
Creating product ttl on: espe larutan penyegar jambu
Creating product ttl on: meiji biscuit hello panda cheese cream
Creating product ttl on: deka mini wafer bites ubi
Creating product ttl on: minute maid juice pulpy orange
Creating product ttl on: indomaret kacang mete madu
Creating product ttl on: better sandwich biscuit vanilla
Creating product ttl on: pringles potato crisps original
Creating product ttl on: dua kelinci kacang sukro kribo
Creating product ttl on: dua kelinci kacang kulit bawang putih
Creating product ttl on: d'veggie sambal tabur original
Creating product ttl on: qtela keripik tem

Creating product ttl on: dilmah green tea jasmine petals 20's
Creating product ttl on: sedaap mie tasty bakmi ayam
Creating product ttl on: lactamil susu bubuk ibu menyusui lactasis vanila
Creating product ttl on: frisian flag susu bubuk instant kompleta chocolate
Creating product ttl on: lactamil susu bubuk ibu hamil pregnasis cokelat
Creating product ttl on: lactamil susu bubuk ibu hamil pregnasis vanila
Creating product ttl on: good mood minuman ekstrak yogurt
Creating product ttl on: roma biscuit sandwich chocolate
Creating product ttl on: nissin wafer ovaltine chocolate malt
Creating product ttl on: sosro teh botol less sugar jasmine tea
Creating product ttl on: nabati richoco wafer chocolate
Creating product ttl on: monde cookies serena egg roll original
Creating product ttl on: monde cookies serena egg roll chocolate
Creating product ttl on: pringles potato crisps seaweed
Creating product ttl on: al shifa natural honey
Creating product ttl on: pringles potato crisps saucy bbq
Cr

Creating product ttl on: roma arden cookies choco splendid
Creating product ttl on: indomilk susu cair uht chocolate
Creating product ttl on: palmia royal butter margarine
Creating product ttl on: silky pudding puding bubuk chocolate
Creating product ttl on: nescafe ice coffee mocha
Creating product ttl on: jetz snack hollow paprika
Creating product ttl on: fitchips multigrain chips cheese & herbs
Creating product ttl on: indomilk kental manis putih
Creating product ttl on: anlene susu bubuk hi-calcium actifit coklat
Creating product ttl on: indomilk kental manis coklat
Creating product ttl on: kartika toast bagelan butter
Creating product ttl on: delfi wafer stick twister minis choco cream
Creating product ttl on: kartika toast bagelan cheese
Creating product ttl on: good time cookies chocochips double choc
Creating product ttl on: nissin crackers crispy
Creating product ttl on: anlene susu bubuk hi-calsium actifit vanila
Creating product ttl on: aqua air mineral click n go kids
Creat

Creating certificate on: 00100100881219
Creating certificate on: 00120038970306
Creating certificate on: 00060020020502
Creating certificate on: 00100100881219
Creating certificate on: 00040029270404
Creating certificate on: 00100082470417
Creating certificate on: 00040040520606
Creating certificate on: 15100045840320
Creating certificate on: 15100045840320
Creating certificate on: 00100062440912
Creating certificate on: 00100014301200
Creating certificate on: 00100014301200
Creating certificate on: 00120000980699
Creating certificate on: 00120035580705
Creating certificate on: 00120035580705
Creating certificate on: 00190083340417
Creating certificate on: 00120035580705
Creating certificate on: 00100014301200
Creating certificate on: 00120017110901
Creating certificate on: 00120035580705
Creating certificate on: 00100024040303
Creating certificate on: 00100017571001
Creating certificate on: 00120071760115
Creating certificate on: 00100110581120
Creating certificate on: 00100014301200


Creating certificate on: 00100029330404
Creating certificate on: 00060070010914
Creating certificate on: 00120065130513
Creating certificate on: 00190019790402
Creating certificate on: 00190019790402
Creating certificate on: 00100084710917
Creating certificate on: 00250001290597
Creating certificate on:  00100069220614
Creating certificate on:  00100069220614
Creating certificate on: 00100017571001
Creating certificate on: 00100039290306
Creating certificate on: 00080022300902
Creating certificate on: 00160097550719
Creating certificate on: 00100000861094
Creating certificate on: 00110080291216
Creating certificate on: 00100040060506
Creating certificate on: 00100024040303
Creating certificate on: 00340080161216
Creating certificate on: 00100017571001
Creating certificate on: 00100051350809
Creating certificate on: 00080007461297
Creating certificate on: 00120071760115
Creating certificate on: 00060070010914
Creating certificate on: 00120071760115
Creating certificate on: 0012007176011

Working on: pt. aromaduta rasaprima
Working on: pt. bandeng juwana
Working on: pt. bina karya prima
Working on: pt. bintang jaya baharriski
Working on: pt. bintang toedjoe
Working on: pt. bumi  sarimas  indonesia
Working on: pt. ceres
Working on: pt. cheetham garam indonesia
Working on: pt. citraraja ampat canning
Working on: pt. coca-cola bottling indonesia
Working on: pt. collita food indonesia
Working on: pt. david roy indonesia
Working on: pt. dcika prima mahkota
Working on: pt. decha mandiri sejahtera
Working on: pt. dima beverages international
Working on: pt. dua kelinci
Working on: pt. eka bogainti
Working on: pt. faretina
Working on: pt. faretina (pabrik di vietnam)
Working on: pt. fonterra brands indonesia
Working on: pt. fonusa agung mulia
Working on: pt. food station tjipinang jaya
Working on: pt. forisa nusapersada
Working on: pt. freyabadi indotama
Working on: pt. frisian flag indonesia
Working on: pt. garudafood putra putri jaya tbk
Working on: pt. gema ista raya
Working

Creating ingredient: zat besi
Creating ingredient: perisa artifisial ayam
Creating ingredient: perisa alamii bawang putih
Creating ingredient: bubuk lada
Creating ingredient: kecap manis
Creating ingredient: pengawet
Creating ingredient: tepung
Creating ingredient: bawang gurih
Creating ingredient: bubuk kakao
Creating ingredient: cokelat mas
Creating ingredient: perisa artifisial cokelat
Creating ingredient: vanili
Creating ingredient: natrium hidrogen karbonat
Creating ingredient: natrium hidroksida
Creating ingredient: enzim papain
Creating ingredient: minyak kanola
Creating ingredient: antioksidan
Creating ingredient: askorbil palmitat
Creating ingredient: serealia
Creating ingredient: tepung oat
Creating ingredient: isomaltulosa
Creating ingredient: sirup gula invert
Creating ingredient: serpihan gandum panggang
Creating ingredient: madu
Creating ingredient: ekstrak malt
Creating ingredient: kelapa parut
Creating ingredient: vitamin
Creating ingredient: milk bubuk
Creating ingredi

Creating ingredient: kalium iodida
Creating ingredient: konsetrat
Creating ingredient: protein whey
Creating ingredient: kaseinat
Creating ingredient: inulin
Creating ingredient: campuran minyak nabati
Creating ingredient: antioksidan dl-alphalfa tokoferol
Creating ingredient: antioksidan natrium askorbat
Creating ingredient: sirup
Creating ingredient: perisa sintetik yogurt
Creating ingredient: frukto oligasakarida
Creating ingredient: yogurt
Creating ingredient: pewarna makanan cokelat
Creating ingredient: lemak reroti nabati
Creating ingredient: cokelat compound butir
Creating ingredient: bumbu rumput laut maltodekstrin
Creating ingredient: dinatrium '-inosinat
Creating ingredient: dinatrium '-guanilat
Creating ingredient: bumbu bbq gula
Creating ingredient: sintetik
Creating ingredient: bubuk susu
Creating ingredient: jagung manis
Creating ingredient: ekstrak ayam
Creating ingredient: kelapa bubuk
Creating ingredient: bumbu garam
Creating ingredient: bumbu rasa ayam
Creating ingred

Creating ingredient: kacang tanah
Creating ingredient: minyak nabati
Creating ingredient: garam
Creating ingredient: penguat rasa
Creating ingredient: gula
Creating ingredient: kacang polong
Creating ingredient: minyak sawit
Creating ingredient: bumbu ayam bawang
Creating ingredient: perisa sintetik daging ayam
Creating ingredient: kecap bubuk
Creating ingredient: ekstrak daging ayam
Creating ingredient: seledri
Creating ingredient: protein nabati terhidrolisis
Creating ingredient: dinatrium guanilat
Creating ingredient: dinatrium inosinat
Creating ingredient: aspartam
Creating ingredient: asesulfam-k
Creating ingredient: pewarna makanan alami karamel
Creating ingredient: whey bubuk
Creating ingredient: natrium bikarbonat
Creating ingredient: mononatrium glutamat
Creating ingredient: daun jeruk purut
Creating ingredient: jagung
Creating ingredient: bumbu bawang putih
Creating ingredient: bawang putih bubuk
Creating ingredient: bawang bombay bubuk
Creating ingredient: susu bubuk instant

Creating ingredient: ikan tuna
Creating ingredient: seafood bubuk
Creating ingredient: lada putih
Creating ingredient: bawang bombay
Creating ingredient: ketumbar
Creating ingredient: pengemulsi pati asetat
Creating ingredient: lada bubuk
Creating ingredient: perisa identik alami bawang putih
Creating ingredient: daun bawang
Creating ingredient: asam folat
Creating ingredient: bumbu rasa telur asin
Creating ingredient: perisa sintetik telur asin
Creating ingredient: pasta kacang
Creating ingredient: whey glukosa
Creating ingredient: pengembang
Creating ingredient: dekstrosa
Creating ingredient: maltitol
Creating ingredient: susu skim cokelat bubuk
Creating ingredient: emping jagung
Creating ingredient: sukralosa
Creating ingredient: cokelat pelapis
Creating ingredient: bumbu rasa korean bbq
Creating ingredient: dekstrosa monohidrat
Creating ingredient: daun teh melati
Creating ingredient: kalium karbonat
Creating ingredient: perisa identik alami
Creating ingredient: rempah-rempah
Creat

Creating ingredient: delima bubuk
Creating ingredient: perisa alamii jeruk nipis
Creating ingredient: brokoli bubuk
Creating ingredient: konsentrat wortel ungu
Creating ingredient: beras bulir panjang
Creating ingredient: beras bulir pendek
Creating ingredient: bubuk kari
Creating ingredient: bumbu kari
Creating ingredient: serbuk gurih kari
Creating ingredient: kopi murni
Creating ingredient: dinatrium inosinat
Creating ingredient: keju bubuk
Creating ingredient: ekstra malt
Creating ingredient: vitamin a
Creating ingredient: pengembang
Creating ingredient: perisa perisa bawang putih
Creating ingredient: saos cabai
Creating ingredient: bawang gurih renyah
Creating ingredient: bumbu nacho cheese mengandung gula
Creating ingredient: bubuk keju
Creating ingredient: bubuk tomat
Creating ingredient: paprika bubuk
Creating ingredient: perisa keju
Creating ingredient: enzim keju termodifikasi
Creating ingredient: perisa daging
Creating ingredient: sayuran
Creating ingredient: penstabil kaliu

Creating ingredient: vitamin a
Creating ingredient: susu bubuk
Creating ingredient: krim strawberry
Creating ingredient: mineral garam
Creating ingredient: konsentrat ari buah apel
Creating ingredient: karboksi metil selulosa
Creating ingredient: artifisial
Creating ingredient: antibuih
Creating ingredient: monogliserida nabati
Creating ingredient: konsentrat sari buah apel
Creating ingredient: konsentrat buah mangga
Creating ingredient: konsetrat sari buah apel
Creating ingredient: identik alami
Creating ingredient: konsetrat sari buah jeruk
Creating ingredient: anti buih
Creating ingredient: monogliserida
Creating ingredient: konsetrat sari buah stroberi
Creating ingredient: humektan
Creating ingredient: natrium asam pirofosfat
Creating ingredient: ekstrak teh
Creating ingredient: konsentrat markisa
Creating ingredient: antioksi
Creating ingredient: asam askorbat
Creating ingredient: lemak susu sapi
Creating ingredient: perisa sintetik stroberi
Creating ingredient: konsetrat stroberi

Creating ingredient: magnesium
Creating ingredient: seng
Creating ingredient: premix vitamin
Creating ingredient: buah nanas
Creating ingredient: bumbu rasa jagung barbeque
Creating ingredient: campuran rempah alami
Creating ingredient: merica
Creating ingredient: perisa buah campuran
Creating ingredient: vitamin mix
Creating ingredient: perisa kafein
Creating ingredient: citrus extract
Creating ingredient: perisa tutti fruity
Creating ingredient: karosen
Creating ingredient: bumbu rawa cabai
Creating ingredient: rempah - rempah
Creating ingredient: konsentrat jambu
Creating ingredient: perisa identik alami jambu
Creating ingredient: nata de coco
Creating ingredient: natrium karboksimetil
Creating ingredient: selulosa
Creating ingredient: gom gelan
